In [58]:
import re
import requests
from bs4 import BeautifulSoup
bacteria_regex = r'[A-Z]\. [a-z]+'
MIN_ID = 1001
MAX_ID = 4202

In [59]:
url_base = 'http://crdd.osdd.net/raghava/hemolytik/display.php?details='
all_bacteria = {}

In [70]:
for j in range(MIN_ID, MAX_ID):
    url = url_base + str(j)
    content = requests.get(url).content
    soup = BeautifulSoup(content, 'html.parser')
    rows = soup.findAll('td')
    sequence, activity, source = None, None, None
    for i, row in enumerate(rows):
        string = row.string
        if row.string:
            if row.string.strip() == 'SEQUENCE':
                sequence = rows[i+1].string.strip()
            elif row.string.strip() == 'ACTIVITY':
                activity = rows[i+1].string.strip()
            elif row.string.strip() == 'RBCs SOURCE':
                source = rows[i+1].string.strip()
    if not sequence:
        print(sequence, j)
    all_bacteria[sequence] = {
        'activity': activity,
        'source': source
    }
    if j % 100 == 0:
        print(j)

(None, 1026)
(None, 1051)
(None, 1056)
(None, 1057)
(None, 1058)
(None, 1059)
(None, 1060)
(None, 1061)
(None, 1062)
(None, 1063)
(None, 1064)
(None, 1065)
(None, 1066)
1100
1200
1300
1400
1500
(None, 1549)
(None, 1550)
(None, 1551)
(None, 1552)
1600
(None, 1613)
(None, 1614)
(None, 1637)
(None, 1638)
(None, 1659)
(None, 1660)
(None, 1661)
(None, 1662)
(None, 1663)
(None, 1664)
(None, 1665)
(None, 1666)
(None, 1667)
(None, 1676)
1700
1800
(None, 1820)
(None, 1821)
(None, 1861)
(None, 1862)
(None, 1863)
(None, 1864)
(None, 1865)
(None, 1869)
(None, 1870)
(None, 1871)
(None, 1880)
(None, 1881)
(None, 1882)
1900
2000
(None, 2034)
(None, 2035)
(None, 2036)
(None, 2037)
(None, 2038)
(None, 2039)
(None, 2040)
(None, 2041)
(None, 2042)
(None, 2043)
(None, 2044)
2100
2200
(None, 2220)
(None, 2221)
(None, 2266)
2300
(None, 2325)
(None, 2328)
(None, 2334)
(None, 2335)
2400
(None, 2417)
(None, 2418)
(None, 2419)
(None, 2453)
(None, 2454)
(None, 2455)
(None, 2456)
(None, 2457)
(None, 2458)
(None, 

In [75]:
all_bacteria

{u'LLRRLLRRLLRRLLRR': {'activity': u'HC50 > ~12', 'source': u'Human'},
 u'KNWKGIAGMAKKLLGENWELM': {'activity': u'5.3% hemolytic at 10\u03bcM',
  'source': u'Human'},
 u'GLPRKILCAIAKKKGKCKGPLKLVCKC': {'activity': u'<1% hemolytic at 200\u03bcM',
  'source': u'Human'},
 u'llgdefrkskekigkefkrivqrikdflrnlvprtes': {'activity': u'HC50 = 125\u03bcg/ml',
  'source': u'Human'},
 u'G-Nle-Aib-Orn-Orn-Nle-Aib-Orn-Orn-Aib-Orn-Orn-GY': {'activity': u'30% hemolysis at 100\u03bcM',
  'source': u'Human'},
 u'GQSQWRDVCRNFMRR': {'activity': u'~40% hemolysis 100\xb5M',
  'source': u'Rabbit'},
 u'KNWKKIASIGKEVLKAL': {'activity': u'IC50 >>100\u03bcM', 'source': u'Rat'},
 u'KWKSFLKTFKSAKKKVLHKALKAISK': {'activity': u'MHC <7.8\u03bcg/ml',
  'source': u'Human'},
 u'VCGETCVGGTCNTPGCSCSRPVCTRNGLP': {'activity': u'50% hemolysis at ~20\u03bcM',
  'source': u'Human'},
 u'KLLKLLLKLLLKLLK': {'activity': u'LD50 =0.3 \u03bcM', 'source': u'Human'},
 u'GLLGPLLKIAKKVGSNLL': {'activity': u'LD50 = 350\xb5M', 'source': u'Huma

u'LC50 =1.4\xb10.2\u03bcM'